In [50]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from keras.utils.vis_utils import plot_model
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [51]:
class google_net(Model):
    def __init__(self):
        super(google_net,self).__init__()

        self.stem=Sequential([   
            layers.Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(7,7),strides=(2,2),padding='same',activation='relu'),
            layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'),   
            layers.LayerNormalization(),
            layers.Conv2D(filters=192,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'),
            layers.Conv2D(filters=192,kernel_size=(3,3),strides=(2,2),padding='same',activation='relu'),
            layers.LayerNormalization(),
            layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')
        ])

        self.l1=Sequential([
            inception(64,96,128,64,64,64),
            inception(64,96,128,64,64,64),
            layers.MaxPooling2D(pool_size=(1,1),strides=(1,1),padding='same')
        ])
        
        self.l2=Sequential([
            inception(64,96,128,64,64,64),
        ])
        self.l3=Sequential([
            inception(64,96,128,64,64,64),
            inception(64,96,128,64,64,64),
            inception(64,96,128,64,64,64)
        ])
        
        self.l4=Sequential([
            inception(64,96,128,64,64,64),
        ])
        
        self.l5=Sequential([
            layers.MaxPooling2D(pool_size=(1,1),strides=(1,1),padding='same'),
            inception(64,96,128,64,64,64),
            inception(64,96,128,64,64,64),
        ])
 
        self.fclayer=Sequential([
            layers.AveragePooling2D(pool_size=(1,1),strides=(1,1)),
            layers.Conv2D(filters=128,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu'),
            layers.Flatten(),
            layers.Dense(units=256,activation='relu'),
            layers.Dropout(0.7),
            layers.Dense(units=10,activation='softmax')
        ])

        self.finalfc=Sequential([
            layers.AveragePooling2D(pool_size=(7,7),strides=(1,1)),
            layers.Flatten(),
            layers.Dense(units=10,activation='softmax')    

        ])

    def call(self,inputs,training=False):
        x=self.stem(inputs,training=training)

        x=self.l1(x,training=False)
        
        x=self.l2(x,training=False)
        ax1=self.fclayer(x,training=False)
        
        x=self.l3(x,training=False)
        ax2=self.fclayer(x,training=False)
        
        x=self.l4(x,training=False)
        
        x=self.l5(x,training=False)
        
        x=self.finalfc(x,training=False)
        return x,ax1,ax2

In [52]:
class inception(layers.Layer):
        def __init__(self,f1,f2,f3,f4,f5,f6):
                super(inception,self).__init__()

                self.layer1=Sequential([
                        layers.Conv2D(filters=f1,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')
                ])
        
                self.layer2=Sequential([
                        layers.Conv2D(filters=f2,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu'),
                        layers.Conv2D(filters=f3,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')

                ])

                self.layer3=Sequential([
                        layers.Conv2D(filters=f4 ,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu'),
                        layers.Conv2D(filters=f5,kernel_size=(5,5),strides=(1,1),padding='same',activation='relu')
                ])

                
                self.layer4=Sequential([
                        layers.MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same'),
                        layers.Conv2D(filters=f6,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')
                ])

        def call(self,x):
                return tf.concat([self.layer1(x),self.layer2(x),self.layer3(x),self.layer4(x)],axis=3)

: 

In [46]:
model=google_net()

In [47]:
model.build(input_shape=(None,224,224,3))

In [48]:
model.summary()

Model: "google_net_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_264 (Sequential)  (None, 14, 14, 192)      452736    
                                                                 
 sequential_273 (Sequential)  (None, 14, 14, 320)      574400    
                                                                 
 sequential_278 (Sequential)  (None, 14, 14, 320)      305632    
                                                                 
 sequential_291 (Sequential)  (None, 14, 14, 320)      916896    
                                                                 
 sequential_296 (Sequential)  (None, 14, 14, 320)      305632    
                                                                 
 sequential_305 (Sequential)  (None, 14, 14, 320)      611264    
                                                                 
 sequential_306 (Sequential)  (None, 10)              

In [ ]:
plot_model(model,to_file='sub.png_subclass',show_shapes=True)